In [296]:
import pandas as pd
import geopandas
import os

In [297]:
df_merge = pd.read_csv("nehnutelnosti_cold_start.csv")
df_schools = pd.read_csv("schools.csv")

In [298]:
# INIT MAP
import folium
from folium.plugins import BeautifyIcon
from folium import plugins
m = folium.Map(location=[48.732681549999995, 21.244332875459712],zoom_start=13)


In [299]:
# Add marker for Boulder, CO
# circle marker

for idx, skola in df_schools.iterrows():
    color = "black"
    if skola["type"] == "elementary":
        color = "green"
    if skola["type"] == "primary":
        color = "blue"
    if skola["type"] == "high":
        color = "red"
    icon_circle = BeautifyIcon(
        icon_shape='circle-dot',
        border_color=color,
        border_width=3,
    )
    folium.Marker(
        location=[skola["y"], skola["x"]], # coordinates for the marker (Earth Lab at CU Boulder)
        popup=skola["organizacia_nazov"], # pop-up label for the marker
        # tooltip='circle',
        icon=icon_circle
    ).add_to(m)

In [300]:
m

In [301]:
# SCHOOOL IN AREA
radius_distance = 750
for idx, item in df_merge.iterrows():
    # print(idx)
    # print(geopandas.points_from_xy(df_merge[idx:idx+1]["x"], df_merge[idx:idx+1]["y"]))
    gdf = geopandas.GeoDataFrame(df_merge[idx:idx+1], geometry=geopandas.points_from_xy(df_merge[idx:idx+1]["x"], df_merge[idx:idx+1]["y"]), crs="EPSG:4326")
    gdf=gdf.to_crs("EPSG:3857") #convert projection web mercator or find best epsg code, where your data have.
    gdf['buffergeometry25miles'] = gdf['geometry'].buffer(radius_distance) # 25 mile = 40233.6 meter radius
    gdf.set_geometry('buffergeometry25miles',inplace=True)
    givengdf = geopandas.GeoDataFrame(df_schools, geometry=geopandas.points_from_xy(df_schools.x, df_schools.y), crs="EPSG:4326")
    givengdf=givengdf.to_crs("EPSG:3857")
    intersect_df = geopandas.overlay(gdf, givengdf, how='intersection', keep_geom_type=False)
    # print(len(intersect_df))
    df_merge.loc[idx, "school_count"] = len(intersect_df)


In [302]:
df_merge

,Unnamed: 0,Unnamed: 0.1,name,productUrl,type,size,singlePrice,state,description,imagePath,...,mhd_count,culture_count,health_lekarne,health_urgent,health_ambu,cycles,school,culture,green,health
0,0,0,PRENÁJOM v NOVOSTAVBE: 2 izb. byt s balónom a ...,https://www.nehnutelnosti.sk/4936736/prenajom-...,2 izbový byt,54.0,"12,59 €/m²/mes.",Novostavba,Ponúkame Vám na prenájom byt v NOVOSTAVBE BYTY...,f2e19e8b-3bc4-41ba-9d98-e5e2d8908383.jpg,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.800,0.000
1,1,1,2 izbový byt v rezidencii PRI RADNICI Strojáre...,https://www.nehnutelnosti.sk/4940194/2-izbovy-...,2 izbový byt,52.0,"16,35 €/m²/mes.",Novostavba,Ponúkame na prenájom kompletne zariadený byt 2...,151bee8d-806f-4218-9df5-facaf099e5fe.jpg,...,0.0,3.0,15.0,6.0,212.0,45.0,4.0,1.5,2.475,5.000
2,2,2,2.izbový byt - dlhodobý prenájom / Michalovce,https://www.nehnutelnosti.sk/4907895/2-izbovy-...,2 izbový byt,54.0,"10,19 €/m²/mes.",Kompletná rekonštrukcia,Dám do dlhodobého prenájmu plne zrekonštruovan...,b45f9eb9-96ab-4e51-bb15-d0219b31789a.jpg,...,1.0,4.0,7.0,3.0,105.0,14.0,2.5,2.0,1.570,2.445
3,3,3,"Slnečný byt, 2 izby + kuchyňa, skvelá lokalita",https://www.nehnutelnosti.sk/4937904/slnecny-b...,2 izbový byt,54.0,"12,59 €/m²/mes.",Čiastočná rekonštrukcia,Máme pre Vás príjemný priestranný 2-izbový byt...,6c22e872-3393-4789-810e-f578cdc0b6d2.jpg,...,2.0,0.0,4.0,0.0,17.0,1.0,1.5,0.0,1.655,0.540
4,4,4,Krásny 2-izbový byt v novostavbe Park Anička,https://www.nehnutelnosti.sk/4912562/krasny-2-...,2 izbový byt,56.0,"14,29 €/m²/mes.",Novostavba,"Ponúkame Vám na prenájom krásny, slnečný, komp...",63e641aa-7ab3-481a-a21e-66967f05ceae.jpg,...,1.0,0.0,1.0,0.0,5.0,2.0,0.0,0.0,0.910,0.135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,100,105,Na prenájom garsónka blízko centra mesta vo vy...,https://www.nehnutelnosti.sk/4908755/na-prenaj...,Garsónka,27.0,"18,52 €/m²/mes.",Čiastočná rekonštrukcia,Ponúkame na prenájom garsónku blízko centra me...,b9d23d1a-060d-4d2a-bbbc-1c014dd7a9c2.jpg,...,3.0,0.0,4.0,0.0,0.0,0.0,1.5,0.0,2.400,0.540
101,101,106,Ponúkame Vám na prenájom 3 - izbový byt s balk...,https://www.nehnutelnosti.sk/4907635/ponukame-...,3 izbový byt,73.0,"7,53 €/m²/mes.",Kompletná rekonštrukcia,Ponúkame Vám na prenájom 3 - izbový byt s balk...,397e2f5b-ae77-4125-8e60-c412d5afca1f.jpg,...,2.0,1.0,5.0,3.0,1.0,20.0,1.5,0.5,2.700,2.175
102,102,107,Veľký (100 m2) 3-izb. BYT S TERASOU v rodinnom...,https://www.nehnutelnosti.sk/4901463/velky-100...,3 izbový byt,100.0,"7,90 €/m²/mes.",Kompletná rekonštrukcia,"""REZERVOVANÉ"" / NA PRENÁJOM: Jedinečný, priest...",6ec91474-a9a7-4e93-8b89-8bae13739255.jpg,...,5.0,0.0,1.0,3.0,2.0,2.0,1.0,0.0,4.110,1.635
103,103,108,Ponúkame Vám na dlhodobý prenájom 2-izbový byt...,https://www.nehnutelnosti.sk/4634888/ponukame-...,2 izbový byt,59.0,"9,32 €/m²/mes.",Kompletná rekonštrukcia,Ponúkame Vám na dlhodobý prenájom 2-izbový byt...,e6eaae77-8ab5-4e67-b826-c63b6fc1c469.jpg,...,3.0,4.0,2.0,3.0,54.0,10.0,3.0,2.0,2.950,1.770


In [303]:
# MHD IN AREA
dataset_path = "_data"
print("DATASET LOADING -> Schools")
filename = "open_ke_mhd.geojson"
file = open(os.path.join(dataset_path,filename))
df_mhd = geopandas.read_file(file)
df_mhd.drop_duplicates(subset='zastavka_nazov', inplace=True)
radius_distance = 500
for idx, item in df_merge.iterrows():
    # print(idx)
    # print(geopandas.points_from_xy(df_merge[idx:idx+1]["x"], df_merge[idx:idx+1]["y"]))
    gdf = geopandas.GeoDataFrame(df_merge[idx:idx+1], geometry=geopandas.points_from_xy(df_merge[idx:idx+1]["x"], df_merge[idx:idx+1]["y"]), crs="EPSG:4326")
    gdf=gdf.to_crs("EPSG:3857") #convert projection web mercator or find best epsg code, where your data have.
    gdf['buffergeometry25miles'] = gdf['geometry'].buffer(radius_distance) # 25 mile = 40233.6 meter radius
    gdf.set_geometry('buffergeometry25miles',inplace=True)
    givengdf = geopandas.GeoDataFrame(df_mhd, geometry=geopandas.points_from_xy(df_mhd.x, df_mhd.y), crs="EPSG:4326")
    givengdf=givengdf.to_crs("EPSG:3857")
    intersect_df = geopandas.overlay(gdf, givengdf, how='intersection', keep_geom_type=False)
    # print(len(intersect_df))
    df_merge.loc[idx, "mhd_count"] = len(intersect_df)


DATASET LOADING -> Schools


In [304]:
# culture IN AREA
dataset_path = "_data"
print("DATASET LOADING -> Schools")
filename = "culture.csv"
df_culture  = pd.read_csv(filename)


radius_distance = 1500
for idx, item in df_merge.iterrows():
    # print(idx)
    # print(geopandas.points_from_xy(df_merge[idx:idx+1]["x"], df_merge[idx:idx+1]["y"]))
    gdf = geopandas.GeoDataFrame(df_merge[idx:idx+1], geometry=geopandas.points_from_xy(df_merge[idx:idx+1]["x"], df_merge[idx:idx+1]["y"]), crs="EPSG:4326")
    gdf=gdf.to_crs("EPSG:3857") #convert projection web mercator or find best epsg code, where your data have.
    gdf['buffergeometry25miles'] = gdf['geometry'].buffer(radius_distance) # 25 mile = 40233.6 meter radius
    gdf.set_geometry('buffergeometry25miles',inplace=True)
    givengdf = geopandas.GeoDataFrame(df_culture, geometry=geopandas.points_from_xy(df_culture.y, df_culture.x), crs="EPSG:4326")
    givengdf=givengdf.to_crs("EPSG:3857")
    intersect_df = geopandas.overlay(gdf, givengdf, how='intersection', keep_geom_type=False)
    # print(len(intersect_df))
    df_merge.loc[idx, "culture_count"] = len(intersect_df)


DATASET LOADING -> Schools


In [305]:
dataset_path = "_data"
sub_folder = "health"
filename = "ambulantne_pohotovosti_ksk.csv"
radius_distance = 3000
df_ambulantne_pohotovost_ksk  = pd.read_csv(os.path.join(dataset_path,sub_folder,filename))
for idx, item in df_merge.iterrows():
    # print(idx)
    # print(geopandas.points_from_xy(df_merge[idx:idx+1]["x"], df_merge[idx:idx+1]["y"]))
    gdf = geopandas.GeoDataFrame(df_merge[idx:idx+1], geometry=geopandas.points_from_xy(df_merge[idx:idx+1]["x"], df_merge[idx:idx+1]["y"]), crs="EPSG:4326")
    gdf=gdf.to_crs("EPSG:3857") #convert projection web mercator or find best epsg code, where your data have.
    gdf['buffergeometry25miles'] = gdf['geometry'].buffer(radius_distance) # 25 mile = 40233.6 meter radius
    gdf.set_geometry('buffergeometry25miles',inplace=True)
    givengdf = geopandas.GeoDataFrame(df_ambulantne_pohotovost_ksk, geometry=geopandas.points_from_xy(df_ambulantne_pohotovost_ksk.poloha_lon, df_ambulantne_pohotovost_ksk.poloha_lat), crs="EPSG:4326")
    givengdf=givengdf.to_crs("EPSG:3857")
    intersect_df = geopandas.overlay(gdf, givengdf, how='intersection', keep_geom_type=False)
    # print(len(intersect_df))
    df_merge.loc[idx, "health_urgent"] = len(intersect_df)

In [306]:
dataset_path = "_data"
sub_folder = "health"
filename = "lekarne_ksk.csv"
radius_distance = 1000
df_lekarne  = pd.read_csv(os.path.join(dataset_path,sub_folder,filename))
for idx, item in df_merge.iterrows():
    # print(idx)
    # print(geopandas.points_from_xy(df_merge[idx:idx+1]["x"], df_merge[idx:idx+1]["y"]))
    gdf = geopandas.GeoDataFrame(df_merge[idx:idx+1], geometry=geopandas.points_from_xy(df_merge[idx:idx+1]["x"], df_merge[idx:idx+1]["y"]), crs="EPSG:4326")
    gdf=gdf.to_crs("EPSG:3857") #convert projection web mercator or find best epsg code, where your data have.
    gdf['buffergeometry25miles'] = gdf['geometry'].buffer(radius_distance) # 25 mile = 40233.6 meter radius
    gdf.set_geometry('buffergeometry25miles',inplace=True)
    givengdf = geopandas.GeoDataFrame(df_lekarne, geometry=geopandas.points_from_xy(df_lekarne.poloha_lon, df_lekarne.poloha_lat), crs="EPSG:4326")
    givengdf=givengdf.to_crs("EPSG:3857")
    intersect_df = geopandas.overlay(gdf, givengdf, how='intersection', keep_geom_type=False)
    # print(len(intersect_df))
    df_merge.loc[idx, "health_lekarne"] = len(intersect_df)

In [307]:
dataset_path = "_data"
sub_folder = "health"
filename = "specializovane_ambulancie_ksk.csv"
df_specializovane_ambulancie_ksk  = pd.read_csv(os.path.join(dataset_path,sub_folder,filename))
# df_prew  = df_specializovane_ambulancie_ksk
# df_specializovane_ambulancie_ksk
radius_distance = 1000

for idx, item in df_merge.iterrows():
    # print(idx)
    # print(geopandas.points_from_xy(df_merge[idx:idx+1]["x"], df_merge[idx:idx+1]["y"]))
    gdf = geopandas.GeoDataFrame(df_merge[idx:idx+1], geometry=geopandas.points_from_xy(df_merge[idx:idx+1]["x"], df_merge[idx:idx+1]["y"]), crs="EPSG:4326")
    gdf=gdf.to_crs("EPSG:3857") #convert projection web mercator or find best epsg code, where your data have.
    gdf['buffergeometry25miles'] = gdf['geometry'].buffer(radius_distance) # 25 mile = 40233.6 meter radius
    gdf.set_geometry('buffergeometry25miles',inplace=True)
    givengdf = geopandas.GeoDataFrame(df_specializovane_ambulancie_ksk, geometry=geopandas.points_from_xy(df_specializovane_ambulancie_ksk.poloha_lon, df_specializovane_ambulancie_ksk.poloha_lat), crs="EPSG:4326")
    givengdf=givengdf.to_crs("EPSG:3857")
    intersect_df = geopandas.overlay(gdf, givengdf, how='intersection', keep_geom_type=False)
    # print(len(intersect_df))
    df_merge.loc[idx, "health_ambu"] = len(intersect_df)

In [308]:
print("DATASET LOADING -> Schools")
filename = "cycle_points.geojson"
file = open(os.path.join(dataset_path,filename))
df_cycle_points = geopandas.read_file(file)
df_cycle_points["y"] = df_cycle_points.apply(lambda row: float(row.geometry[0].y), axis=1)
df_cycle_points["x"] = df_cycle_points.apply(lambda row: float(row.geometry[0].x), axis=1)
radius_distance = 1000

for idx, item in df_merge.iterrows():
    # print(idx)
    # print(geopandas.points_from_xy(df_merge[idx:idx+1]["x"], df_merge[idx:idx+1]["y"]))
    gdf = geopandas.GeoDataFrame(df_merge[idx:idx+1], geometry=geopandas.points_from_xy(df_merge[idx:idx+1]["x"], df_merge[idx:idx+1]["y"]), crs="EPSG:4326")
    gdf=gdf.to_crs("EPSG:3857") #convert projection web mercator or find best epsg code, where your data have.
    gdf['buffergeometry25miles'] = gdf['geometry'].buffer(radius_distance) # 25 mile = 40233.6 meter radius
    gdf.set_geometry('buffergeometry25miles',inplace=True)
    givengdf = geopandas.GeoDataFrame(df_cycle_points, geometry=geopandas.points_from_xy(df_cycle_points["x"],df_cycle_points["y"], crs="EPSG:4326"))
    givengdf=givengdf.to_crs("EPSG:3857")
    intersect_df = geopandas.overlay(gdf, givengdf, how='intersection', keep_geom_type=False)
    # print(len(intersect_df))
    df_merge.loc[idx, "cycles"] = len(intersect_df)

DATASET LOADING -> Schools


/var/folders/mt/zbsgqls91d5c1dt725ckq4lc0000gn/T/ipykernel_26318/742432914.py:5: ShapelyDeprecationWarning: __getitem__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  df_cycle_points["y"] = df_cycle_points.apply(lambda row: float(row.geometry[0].y), axis=1)
/var/folders/mt/zbsgqls91d5c1dt725ckq4lc0000gn/T/ipykernel_26318/742432914.py:6: ShapelyDeprecationWarning: __getitem__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  df_cycle_points["x"] = df_cycle_points.apply(lambda row: float(row.geometry[0].x), axis=1)


In [309]:
df_merge

,Unnamed: 0,Unnamed: 0.1,name,productUrl,type,size,singlePrice,state,description,imagePath,...,mhd_count,culture_count,health_lekarne,health_urgent,health_ambu,cycles,school,culture,green,health
0,0,0,PRENÁJOM v NOVOSTAVBE: 2 izb. byt s balónom a ...,https://www.nehnutelnosti.sk/4936736/prenajom-...,2 izbový byt,54.0,"12,59 €/m²/mes.",Novostavba,Ponúkame Vám na prenájom byt v NOVOSTAVBE BYTY...,f2e19e8b-3bc4-41ba-9d98-e5e2d8908383.jpg,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.800,0.000
1,1,1,2 izbový byt v rezidencii PRI RADNICI Strojáre...,https://www.nehnutelnosti.sk/4940194/2-izbovy-...,2 izbový byt,52.0,"16,35 €/m²/mes.",Novostavba,Ponúkame na prenájom kompletne zariadený byt 2...,151bee8d-806f-4218-9df5-facaf099e5fe.jpg,...,0.0,3.0,15.0,6.0,212.0,45.0,4.0,1.5,2.475,5.000
2,2,2,2.izbový byt - dlhodobý prenájom / Michalovce,https://www.nehnutelnosti.sk/4907895/2-izbovy-...,2 izbový byt,54.0,"10,19 €/m²/mes.",Kompletná rekonštrukcia,Dám do dlhodobého prenájmu plne zrekonštruovan...,b45f9eb9-96ab-4e51-bb15-d0219b31789a.jpg,...,1.0,4.0,7.0,3.0,105.0,14.0,2.5,2.0,1.570,2.445
3,3,3,"Slnečný byt, 2 izby + kuchyňa, skvelá lokalita",https://www.nehnutelnosti.sk/4937904/slnecny-b...,2 izbový byt,54.0,"12,59 €/m²/mes.",Čiastočná rekonštrukcia,Máme pre Vás príjemný priestranný 2-izbový byt...,6c22e872-3393-4789-810e-f578cdc0b6d2.jpg,...,2.0,0.0,4.0,0.0,17.0,1.0,1.5,0.0,1.655,0.540
4,4,4,Krásny 2-izbový byt v novostavbe Park Anička,https://www.nehnutelnosti.sk/4912562/krasny-2-...,2 izbový byt,56.0,"14,29 €/m²/mes.",Novostavba,"Ponúkame Vám na prenájom krásny, slnečný, komp...",63e641aa-7ab3-481a-a21e-66967f05ceae.jpg,...,1.0,0.0,1.0,0.0,5.0,2.0,0.0,0.0,0.910,0.135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,100,105,Na prenájom garsónka blízko centra mesta vo vy...,https://www.nehnutelnosti.sk/4908755/na-prenaj...,Garsónka,27.0,"18,52 €/m²/mes.",Čiastočná rekonštrukcia,Ponúkame na prenájom garsónku blízko centra me...,b9d23d1a-060d-4d2a-bbbc-1c014dd7a9c2.jpg,...,3.0,0.0,4.0,0.0,0.0,0.0,1.5,0.0,2.400,0.540
101,101,106,Ponúkame Vám na prenájom 3 - izbový byt s balk...,https://www.nehnutelnosti.sk/4907635/ponukame-...,3 izbový byt,73.0,"7,53 €/m²/mes.",Kompletná rekonštrukcia,Ponúkame Vám na prenájom 3 - izbový byt s balk...,397e2f5b-ae77-4125-8e60-c412d5afca1f.jpg,...,2.0,1.0,5.0,3.0,1.0,20.0,1.5,0.5,2.700,2.175
102,102,107,Veľký (100 m2) 3-izb. BYT S TERASOU v rodinnom...,https://www.nehnutelnosti.sk/4901463/velky-100...,3 izbový byt,100.0,"7,90 €/m²/mes.",Kompletná rekonštrukcia,"""REZERVOVANÉ"" / NA PRENÁJOM: Jedinečný, priest...",6ec91474-a9a7-4e93-8b89-8bae13739255.jpg,...,5.0,0.0,1.0,3.0,2.0,2.0,1.0,0.0,4.110,1.635
103,103,108,Ponúkame Vám na dlhodobý prenájom 2-izbový byt...,https://www.nehnutelnosti.sk/4634888/ponukame-...,2 izbový byt,59.0,"9,32 €/m²/mes.",Kompletná rekonštrukcia,Ponúkame Vám na dlhodobý prenájom 2-izbový byt...,e6eaae77-8ab5-4e67-b826-c63b6fc1c469.jpg,...,3.0,4.0,2.0,3.0,54.0,10.0,3.0,2.0,2.950,1.770


In [310]:
# SCHOOOL
df_merge["school"] = df_merge["school_count"]
df_merge.loc[df_merge["school"] >= 10, "school"] = 10
df_merge["school"] = df_merge["school"] / 2

# CULTURE
df_merge["culture"] = df_merge["culture_count"]
df_merge.loc[df_merge["culture"] >= 10, "culture"] = 10
df_merge["culture"] = df_merge["culture"] / 2

# GREEN_TRANSPORT
df_merge["green"] = df_merge["mhd_count"] * 0.8 + df_merge["cycles"] / 10 * 0.55
df_merge.loc[df_merge["green"] >= 5, "green"] = 5
# GREEN_TRANSPORT
df_merge["health"] = df_merge["health_urgent"] * 0.5 + df_merge["health_lekarne"] / 2 * 0.25 + df_merge["health_lekarne"] / 100
df_merge.loc[df_merge["health"] >= 5, "health"] = 5

In [311]:
df_merge.to_csv("nehnutelnosti_cold_start.csv")

In [312]:
# # Add marker for Boulder, CO
# # circle marker
#
# for idx, skola in df_schools.loc[df_schools["space_code_id"].isin(intersect_df["space_code_id"])].iterrows():
#     color = "black"
#     if skola["type"] == "elementary":
#         color = "green"
#     if skola["type"] == "primary":
#         color = "blue"
#     if skola["type"] == "high":
#         color = "red"
#     icon_circle = BeautifyIcon(
#         icon_shape='circle-dot',
#         border_color=color,
#         border_width=5,
#     )
#     folium.Marker(
#         location=[skola["y"], skola["x"]], # coordinates for the marker (Earth Lab at CU Boulder)
#         popup=skola["organizacia_nazov"], # pop-up label for the marker
#         # tooltip='circle',
#         icon=icon_circle
#     ).add_to(m)

In [313]:
# Add marker for Boulder, CO
# circle marker

for idx, byt in df_merge.iterrows():
    color = "black"

    icon_circle = BeautifyIcon(
        icon_shape='circle-dot',
        border_color=color,
        border_width=3,
    )
    folium.Marker(
        location=[byt["y"], byt["x"]], # coordinates for the marker (Earth Lab at CU Boulder)
        popup=str(byt["school_count"]), # pop-up label for the marker
        # tooltip='circle',
        icon=icon_circle
    ).add_to(m)

In [314]:
m